# LangChain Expression Language Cheatsheet

This is a quick reference for all the most important LCEL primitives. For more advanced usage see the [LCEL how-to guides](/docs/how_to/#langchain-expression-language-lcel) and the [full API reference](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html).

### How to invoke a runnable: `Runnable.invoke()`

In [6]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(lambda x: str(x))
runnable.invoke(5)

# Async variant:
# await runnable.ainvoke(5)

'5'

### How to batch a runnable: `Runnable.batch()`

In [7]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(lambda x: str(x))
runnable.batch([7, 8, 9])

# Async variant:
# await runnable.abatch([7, 8, 9])

['7', '8', '9']

### How to stream a runnable: `Runnable.stream()`

In [8]:
from langchain_core.runnables import RunnableLambda


def func(x):
    for y in x:
        yield str(y)


runnable = RunnableLambda(func)

for chunk in runnable.stream(range(5)):
    print(chunk)

# Async variant:
# async for chunk in await runnable.astream(range(5)):
#     print(chunk)

0
1
2
3
4


### How to compose runnables: `|`

In [10]:
from langchain_core.runnables import RunnableLambda

runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 2)

chain = runnable1 | runnable2

chain.invoke(2)

[{'foo': 2}, {'foo': 2}]

### How to invoke runnables in parallel: `RunnableParallel`

In [11]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 2)

chain = RunnableParallel(first=runnable1, second=runnable2)

chain.invoke(2)

{'first': {'foo': 2}, 'second': [2, 2]}

### How to turn any function into a runnable: `RunnableLambda`

In [23]:
from langchain_core.runnables import RunnableLambda


def func(x):
    return x + 5


runnable = RunnableLambda(func)
runnable.invoke(2)

7

### How to merge input and output dicts: `RunnablePassthrough.assign`

In [13]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

runnable1 = RunnableLambda(lambda x: x["foo"] + 7)

chain = RunnablePassthrough.assign(bar=runnable1)

chain.invoke({"foo": 10})

{'foo': 10, 'bar': 17}

### How to include input dict in output dict: `RunnablePassthrough`

In [15]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

runnable1 = RunnableLambda(lambda x: x["foo"] + 7)

chain = RunnableParallel(bar=runnable1, baz=RunnablePassthrough())

chain.invoke({"foo": 10})

{'bar': 17, 'baz': {'foo': 10}}

### How to add fallbacks: `Runnable.with_fallbacks`

In [19]:
from langchain_core.runnables import RunnableLambda

runnable1 = RunnableLambda(lambda x: x + "foo")
runnable2 = RunnableLambda(lambda x: str(x) + "foo")

chain = runnable1.with_fallbacks([runnable2])

chain.invoke(5)

'5foo'

### How to add retries: `Runnable.with_retries`

In [32]:
from langchain_core.runnables import RunnableLambda

counter = -1


def func(x):
    global counter
    counter += 1
    print(f"attempt with {counter=}")
    return x / counter


chain = RunnableLambda(func).with_retry(stop_after_attempt=2)

chain.invoke(2)

attempt with counter=0
attempt with counter=1


2.0

### How to return subset of input dict: `Runnable.pick`

### How to declaratively make a batched version of a runnable: `Runnable.map`

In [20]:
from langchain_core.runnables import RunnableLambda

runnable1 = RunnableLambda(lambda x: x + 5)
runnable2 = RunnableLambda(lambda x: list(range(x)))

chain = runnable2 | runnable1.map()

chain.invoke(3)

[5, 6, 7]

### How to configure runnable params: `Runnable.with_configurable_fields`

### How to configure runnable alternatives: `Runnable.with_configurable_alternatives`

### How to build a chain dynamically

### How to generate a stream of events: `Runnable.stream_events`

### How to yield batched outputs as they complete: `Runnable.batch_as_completed`

### How to get a graph representation of a runnable: `Runnable.get_graph`

### How to get all prompts in a runnable: `Runnable.get_prompts